<a href="https://colab.research.google.com/github/dan-candeira/Nuforc-Project/blob/master/5.7-Preparacao/5.7-nuforc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
! pip install pandasql
import pandasql as ps

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26820 sha256=703fe31e6c226d3b2fbb6f3512aea62a037bb74fe3df37ac134fa621e8136ee9
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/dan-candeira/Nuforc-Project/master/5.2-Coleta/ufoCollection.csv')\
.drop(columns = ['Unnamed: 0'])

In [0]:
df.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01


In [0]:
query = '''
  SELECT * FROM df
  WHERE State != ''
  AND LOWER(State) != 'none'
  AND LOWER(State) != 'unknown'
  AND Shape != ''
  AND LOWER(Shape) != 'none'
  AND LOWER(Shape) != 'unknown'
  AND City != ''
  AND LOWER(City) != 'none'
  AND LOWER(City) != 'unknown'
  '''
clean_df = ps.sqldf(query, locals())

In [0]:
clean_df.head()

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,9/30/97 22:00,Madison,WI,Light,5 minutes,Strange light inside Lake Monona,3/2/04
1,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
2,9/28/97 23:15,San Francisco,CA,Triangle,12-15s,flying-wing shape outlined by 12-14 lights. Ap...,7/5/99
3,9/27/97 23:00,Egan,SD,Other,30 minutes,The Weirdest Thing I Have Ever Seen,2/22/05
4,9/27/97 05:00,Crestwood,KY,Disk,15 minutes,A big disk with red and green lights on the ri...,8/5/01


In [0]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84029 entries, 0 to 84028
Data columns (total 7 columns):
Date / Time    84029 non-null object
City           84029 non-null object
State          84029 non-null object
Shape          84029 non-null object
Duration       82026 non-null object
Summary        84020 non-null object
Posted         84027 non-null object
dtypes: object(7)
memory usage: 4.5+ MB


In [0]:
us_states = pd.read_csv('https://raw.githubusercontent.com/dan-candeira/Nuforc-Project/master/5.7-Preparacao/states.csv', sep=';')

In [0]:
us_states.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [0]:
query = '''
  SELECT clean_df.* FROM clean_df, us_states
  WHERE clean_df.State = Abbreviation
'''
us_df = ps.sqldf(query, locals())

In [0]:
us_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80159 entries, 0 to 80158
Data columns (total 7 columns):
Date / Time    80159 non-null object
City           80159 non-null object
State          80159 non-null object
Shape          80159 non-null object
Duration       78266 non-null object
Summary        80150 non-null object
Posted         80157 non-null object
dtypes: object(7)
memory usage: 4.3+ MB


In [0]:
new_df = us_df.drop(columns=['Posted', 'Duration', 'Summary'])

In [0]:
new_df.head()

,Date / Time,City,State,Shape
0,9/30/97 22:00,Madison,WI,Light
1,9/28/97 23:15,San Francisco,CA,Triangle
2,9/27/97 23:00,Egan,SD,Other
3,9/27/97 05:00,Crestwood,KY,Disk
4,9/25/97 22:00,Clearfield,UT,Triangle


In [0]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80159 entries, 0 to 80158
Data columns (total 4 columns):
Date / Time    80159 non-null object
City           80159 non-null object
State          80159 non-null object
Shape          80159 non-null object
dtypes: object(4)
memory usage: 2.4+ MB


In [0]:
query = '''
  SELECT Shape, COUNT(Shape) as views FROM new_df
  GROUP BY Shape
  HAVING views > 1000
'''

popular_shapes = ps.sqldf(query, locals())

In [0]:
popular_shapes.head()

,Shape,views
0,Changing,2275
1,Chevron,1041
2,Cigar,1896
3,Circle,9313
4,Cylinder,1367


In [0]:
popular_shapes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
Shape    16 non-null object
views    16 non-null int64
dtypes: int64(1), object(1)
memory usage: 384.0+ bytes


In [0]:
query = '''
  SELECT new_df.* FROM new_df, popular_shapes
  WHERE new_df.Shape = popular_shapes.Shape
'''
final_df = ps.sqldf(query, locals())

In [0]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77902 entries, 0 to 77901
Data columns (total 4 columns):
Date / Time    77902 non-null object
City           77902 non-null object
State          77902 non-null object
Shape          77902 non-null object
dtypes: object(4)
memory usage: 2.4+ MB


In [0]:
final_df.to_csv('df_OVNI_limpo.csv')